In [1]:
%matplotlib inline

# from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import math

Using Theano backend.
/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7003 on context None
Mapped name None to device cuda: Tesla K80 (0000:00:1E.0)


In [2]:
import utils; 
#from urllib import reload
#reload(utils)
from utils import plots

In [3]:
if 'BASE_DIR' not in locals():
    BASE_DIR = os.getcwd()
print(BASE_DIR)

/home/ubuntu/data_drive/fast.ai_oren/courses/deeplearning1/nbs


# Download and unzip data

In [4]:
DATA_BASE_DIR = os.path.join(BASE_DIR, 'data/dogscatskaggle')

%mkdir -p $DATA_BASE_DIR


In [ ]:
%cd $DATA_BASE_DIR
os.system("kg download -c dogs-vs-cats-redux-kernels-edition")
os.system("unzip -q train.zip")
os.system("unzip -q test.zip")
%cd $BASE_DIR

In [ ]:
train_path = os.path.join(DATA_BASE_DIR, "train")
filenames = os.listdir(train_path)
cats_path = os.path.join(train_path, "cats")
dogs_path = os.path.join(train_path, "dogs")
if not os.path.exists(cats_path):
    os.makedirs(cats_path)
if not os.path.exists(dogs_path):
    os.makedirs(dogs_path)
        
for filename in filenames:
    if "cat" in filename:
        os.rename(os.path.join(train_path, filename), os.path.join(cats_path, filename[4:]))
    elif "dog" in filename:
        os.rename(os.path.join(train_path, filename), os.path.join(dogs_path, filename[4:]))


# Prepare train, validation, sample data

In [5]:
import sys
sys.path.insert(0, '/home/ubuntu/data_drive')
sys.path.insert(0, '/home/ubuntu/data_drive/deeputils')
import deeputils


In [8]:
print(DATA_BASE_DIR)
deeputils.file.split_train(DATA_BASE_DIR)
deeputils.file.gen_sample(DATA_BASE_DIR, split=0.05)

/home/ubuntu/data_drive/fast.ai_oren/courses/deeplearning1/nbs/data/dogscatskaggle
number of items to move:  0
src_dir: /home/ubuntu/data_drive/fast.ai_oren/courses/deeplearning1/nbs/data/dogscatskaggle/train/cats number of items to copy: 0
src_dir: /home/ubuntu/data_drive/fast.ai_oren/courses/deeplearning1/nbs/data/dogscatskaggle/train/dogs number of items to copy: 0
src_dir: /home/ubuntu/data_drive/fast.ai_oren/courses/deeplearning1/nbs/data/dogscatskaggle/test number of items to copy: 0
src_dir: /home/ubuntu/data_drive/fast.ai_oren/courses/deeplearning1/nbs/data/dogscatskaggle/valid/cats number of items to copy: 0
src_dir: /home/ubuntu/data_drive/fast.ai_oren/courses/deeplearning1/nbs/data/dogscatskaggle/valid/dogs number of items to copy: 0


## pre-fetch images and save as one large array

In [6]:
path = "data/dogscatskaggle/"
# path = "data/dogscatskaggle/sample/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size = 64

In [7]:
# data_train = deeptuils.imageutils.get_data(path+ 'train')
# data_valid = deeptuils.imageutils.get_data(path+ 'valid')
# print(data_train.shape)
# print(data_valid.shape)

In [8]:
batches_train = image.ImageDataGenerator().flow_from_directory(path+'train', shuffle=False, batch_size=batch_size, 
                                         class_mode='categorical', target_size=(224, 224))
batches_valid = image.ImageDataGenerator().flow_from_directory(path+'valid', shuffle=False, batch_size=batch_size, 
                                         class_mode='categorical', target_size=(224, 224))
batches_test = image.ImageDataGenerator().flow_from_directory(path+'test', shuffle=False, batch_size=batch_size, 
                                         class_mode='categorical', target_size=(224, 224))
steps_train = math.ceil(batches_train.samples / batch_size)
steps_valid = math.ceil(batches_valid.samples / batch_size)
steps_test = math.ceil(batches_test.samples / batch_size)
print(steps_train)
print(steps_valid)
print(steps_test)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
352
40
196


In [9]:
classes_train = batches_train.classes
classes_valid = batches_valid.classes
classes_test = batches_test.classes
labels_train = deeputils.general.onehot(classes_train)
labels_valid = deeputils.general.onehot(classes_valid)
labels_test = deeputils.general.onehot(classes_test)

## pass datasets through pre-trained convolution layers

In [11]:
import keras
from keras.applications.vgg19 import VGG19
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Model

input_tensor = Input(shape=(3, 224,224))
model_suffix = "_vgg19"
conv_model = VGG19(include_top=False, weights='imagenet', input_tensor=input_tensor)

for layer in conv_model.layers:
    layer.trainable = False


In [12]:
conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 224, 224)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 224, 224)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 112, 112)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 112, 112)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 112, 112)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 56, 56)       0         
__________

In [12]:
features_train = conv_model.predict_generator(batches_train, steps_train)
print(features_train.shape)
features_valid = conv_model.predict_generator(batches_valid, steps_valid)
print(features_valid.shape)
features_test = conv_model.predict_generator(batches_test, steps_test)
print(features_test.shape)

(22500, 512, 7, 7)
(2500, 512, 7, 7)
(12500, 512, 7, 7)


In [13]:
deeputils.file.save_array(model_path + 'train_convlayers_features'+model_suffix+'.bc', features_train)
deeputils.file.save_array(model_path + 'valid_convlayers_features'+model_suffix+'.bc', features_valid)
deeputils.file.save_array(model_path + 'test_convlayers_features'+model_suffix+'.bc', features_test)

# train fc layers

In [14]:
features_train = deeputils.file.load_array(model_path + 'train_convlayers_features'+model_suffix+'.bc')
features_valid = deeputils.file.load_array(model_path + 'valid_convlayers_features'+model_suffix+'.bc')
features_test = deeputils.file.load_array(model_path + 'test_convlayers_features'+model_suffix+'.bc')
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(22500, 512, 7, 7)
(2500, 512, 7, 7)
(12500, 512, 7, 7)


In [19]:
from keras.optimizers import RMSprop, Adam
from keras.models import Sequential
from keras.layers import MaxPooling2D, Flatten, Dense, BatchNormalization

#def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

# opt = Adam()
def get_fc_model(dropout, lr):
    model = Sequential([
        MaxPooling2D(input_shape=conv_model.layers[-1].output_shape[1:]),
#         Flatten(input_shape=conv_model.layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
#         BatchNormalization(),
        Dropout(dropout),
        Dense(4096, activation='relu'),
#         BatchNormalization(),
        Dropout(dropout),
        Dense(2, activation='softmax')
        ])

#     opt = RMSprop(lr=lr, rho=0.7)
    opt = Adam(lr=lr)
    
#     for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(l2.get_weights())

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [72]:
fc_model05 = get_fc_model(0.5, 0.0001)
fc_model05.fit(features_train, labels_train, epochs = 5, batch_size=batch_size,
             validation_data = (features_valid, labels_valid))


Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 6s - loss: 1.5969 - acc: 0.8936 - val_loss: 1.3853 - val_acc: 0.9116
Epoch 2/5
22500/22500 [==============================] - 6s - loss: 0.8706 - acc: 0.9428 - val_loss: 0.5226 - val_acc: 0.9652
Epoch 3/5
22500/22500 [==============================] - 6s - loss: 0.7581 - acc: 0.9511 - val_loss: 0.5899 - val_acc: 0.9612
Epoch 4/5
22500/22500 [==============================] - 6s - loss: 0.7440 - acc: 0.9515 - val_loss: 0.6083 - val_acc: 0.9612
Epoch 5/5
22500/22500 [==============================] - 6s - loss: 0.7489 - acc: 0.9522 - val_loss: 0.7554 - val_acc: 0.9520


In [16]:
fc_model05.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
batch_normalization_3 (Batch (None, 4096)              16384     
_________________________________________________________________
dropout_3 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
batch_normalization_4 (Batch (None, 4096)              16384     
_________________________________________________________________
dropout_4 (Dropout)          (None, 4096)              0         
__________

In [22]:
fc_model00 = get_fc_model(0.2, 0.00001)
fc_model00.fit(features_train, labels_train, epochs = 5, batch_size=batch_size,
             validation_data = (features_valid, labels_valid))


Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 7s - loss: 0.6876 - acc: 0.9331 - val_loss: 0.3793 - val_acc: 0.9660
Epoch 2/5
22500/22500 [==============================] - 7s - loss: 0.3371 - acc: 0.9683 - val_loss: 0.3065 - val_acc: 0.9696
Epoch 3/5
22500/22500 [==============================] - 7s - loss: 0.2614 - acc: 0.9759 - val_loss: 0.4492 - val_acc: 0.9616
Epoch 4/5
22500/22500 [==============================] - 7s - loss: 0.2340 - acc: 0.9776 - val_loss: 0.2974 - val_acc: 0.9724
Epoch 5/5
22500/22500 [==============================] - 7s - loss: 0.1883 - acc: 0.9825 - val_loss: 0.3155 - val_acc: 0.9740


In [23]:
fc_model00.optimizer.lr = 0.000001
fc_model00.fit(features_train, labels_train, epochs = 5, batch_size=batch_size,
             validation_data = (features_valid, labels_valid))


Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 7s - loss: 0.1631 - acc: 0.9847 - val_loss: 0.3686 - val_acc: 0.9692
Epoch 2/5
22500/22500 [==============================] - 7s - loss: 0.1472 - acc: 0.9859 - val_loss: 0.3099 - val_acc: 0.9756
Epoch 3/5
22500/22500 [==============================] - 7s - loss: 0.1132 - acc: 0.9898 - val_loss: 0.2937 - val_acc: 0.9740
Epoch 4/5
22500/22500 [==============================] - 7s - loss: 0.1123 - acc: 0.9895 - val_loss: 0.3735 - val_acc: 0.9660
Epoch 5/5
22500/22500 [==============================] - 7s - loss: 0.1004 - acc: 0.9907 - val_loss: 0.2930 - val_acc: 0.9720


In [24]:
fc_model00.optimizer.lr = 0.0000001
fc_model00.fit(features_train, labels_train, epochs = 5, batch_size=batch_size,
             validation_data = (features_valid, labels_valid))


Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 7s - loss: 0.0903 - acc: 0.9917 - val_loss: 0.3425 - val_acc: 0.9692
Epoch 2/5
22500/22500 [==============================] - 7s - loss: 0.0769 - acc: 0.9933 - val_loss: 0.3252 - val_acc: 0.9684
Epoch 3/5
22500/22500 [==============================] - 7s - loss: 0.0885 - acc: 0.9924 - val_loss: 0.3216 - val_acc: 0.9724
Epoch 4/5
22500/22500 [==============================] - 7s - loss: 0.0675 - acc: 0.9939 - val_loss: 0.3386 - val_acc: 0.9724
Epoch 5/5
22500/22500 [==============================] - 7s - loss: 0.0640 - acc: 0.9941 - val_loss: 0.3307 - val_acc: 0.9720


In [248]:
fc_model00.optimizer.lr = 0.0001
fc_model00.fit(features_train, labels_train, epochs = 5, batch_size=batch_size,
             validation_data = (features_valid, labels_valid))


Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 6s - loss: 1.3644 - acc: 0.9152 - val_loss: 0.9735 - val_acc: 0.9396
Epoch 2/5
22500/22500 [==============================] - 6s - loss: 1.2372 - acc: 0.9232 - val_loss: 0.9800 - val_acc: 0.9392
Epoch 3/5
22500/22500 [==============================] - 6s - loss: 1.1953 - acc: 0.9258 - val_loss: 0.9285 - val_acc: 0.9424
Epoch 4/5
22500/22500 [==============================] - 6s - loss: 1.4199 - acc: 0.9119 - val_loss: 1.3232 - val_acc: 0.9176
Epoch 5/5
22500/22500 [==============================] - 6s - loss: 1.3137 - acc: 0.9185 - val_loss: 0.9408 - val_acc: 0.9416


In [190]:
# fc_model00.save_weights(path+'no_dropout.h5')
# fc_model00.save(path+'no_dropout_model.hdf5')

In [17]:
model = Model(inputs=conv_model.input, outputs=fc_model00(conv_model.output))
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.save_weights(model_path+'no_dropout.h5')


In [18]:
model_json = model.to_json()
with open(model_path+"no_dropout_model.json", "w") as json_file:
    json_file.write(model_json)


# Augmentation

In [76]:
for layer in conv_model.layers: 
    layer.trainable = False
print(type(conv_model.layers[0]))
model = Model(inputs=conv_model.input, outputs=fc_model00(conv_model.output))
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

<class 'keras.engine.topology.InputLayer'>


In [77]:
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)
batches_aug_train = gen.flow_from_directory(path+'train', shuffle=True, batch_size=batch_size, 
                                         class_mode='categorical', target_size=(224, 224))
batches_aug_valid = gen.flow_from_directory(path+'valid', shuffle=True, batch_size=batch_size, 
                                         class_mode='categorical', target_size=(224, 224))


Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [78]:
model.fit_generator(batches_aug_train, steps_per_epoch=steps_train, epochs=3, 
                              validation_data=batches_aug_valid, validation_steps=steps_valid)

Epoch 1/3
352/352 [==============================] - 516s - loss: 1.5751 - acc: 0.8983 - val_loss: 1.2071 - val_acc: 0.9216
Epoch 2/3
352/352 [==============================] - 516s - loss: 0.9987 - acc: 0.9364 - val_loss: 0.7595 - val_acc: 0.9512
Epoch 3/3
352/352 [==============================] - 516s - loss: 1.0292 - acc: 0.9348 - val_loss: 2.5446 - val_acc: 0.8412


In [79]:
model.fit_generator(batches_aug_train, steps_per_epoch=steps_train, epochs=3, 
                              validation_data=batches_aug_valid, validation_steps=steps_valid)


Epoch 1/3
351/352 [============================>.] - ETA: 1s - loss: 1.1082 - acc: 0.9303

KeyboardInterrupt: 

In [58]:
conv_model.save_weights(path+'augmented.h5')
conv_model.save(path+'augmented_model.hdf5')

# submit predictions

In [19]:
from keras.models import model_from_json

json_file = open(model_path + 'no_dropout_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
submit_model = model_from_json(loaded_model_json)
# load weights into new model
submit_model.load_weights(model_path + "no_dropout.h5")


In [63]:
submit_model = fc_model00

In [73]:
# predictions = submit_model.predict_generator(batches_test, steps_test)


In [74]:
predictions = fc_model00.predict(features_test, steps_test)

In [75]:
print(predictions.shape)
print(predictions[:5])

(12500, 2)
[[  1.2579e-32   1.0000e+00]
 [  1.0000e+00   1.7184e-41]
 [  3.1085e-34   1.0000e+00]
 [  0.0000e+00   1.0000e+00]
 [  1.0000e+00   0.0000e+00]]


In [76]:
filenames = batches_test.filenames
print(filenames[:5])


['unknown/2212.jpg', 'unknown/11615.jpg', 'unknown/9910.jpg', 'unknown/6117.jpg', 'unknown/541.jpg']


In [77]:
isdog = predictions[:,1]
print(isdog[:5])

[  1.0000e+00   1.7184e-41   1.0000e+00   1.0000e+00   0.0000e+00]


In [78]:
margin = 0.04
isdog = isdog.clip(min=margin, max=1-margin)
print(isdog[:5])

[ 0.96  0.04  0.96  0.96  0.04]


In [79]:
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
print(ids)

[ 2212 11615  9910 ...,     6  7220  4813]


In [80]:
subm = np.stack([ids,isdog], axis=1)
subm[:5]


array([[  2.2120e+03,   9.6000e-01],
       [  1.1615e+04,   4.0000e-02],
       [  9.9100e+03,   9.6000e-01],
       [  6.1170e+03,   9.6000e-01],
       [  5.4100e+02,   4.0000e-02]])

In [81]:
submission_file_name = 'submission1.csv'
np.savetxt(model_path + submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

In [56]:
from IPython.display import FileLink

FileLink(model_path + submission_file_name)


/home/ubuntu/data_drive/fast.ai_oren/courses/deeplearning1/nbs/data/dogscatskaggle/models/submission1.csv

In [24]:
features_test = conv_model.predict_generator(batches_test, steps_test)
print(features_test.shape)


(12500, 512, 7, 7)


In [25]:
deeputils.file.save_array(model_path + 'test_convlayers_features.bc', features_test)


In [12]:
features_test = deeputils.file.load_array(model_path + 'test_convlayers_features.bc')
print(features_test.shape)


(12500, 512, 7, 7)


# Ensambling

In [25]:
def fit_model():
    model = Sequential([
#         MaxPooling2D(input_shape=conv_model.layers[-1].output_shape[1:]),
        Flatten(input_shape=conv_model.layers[-1].output_shape[1:]),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(2, activation='softmax')
        ])

#     opt = RMSprop(lr=lr, rho=0.7)
    opt = Adam(lr=0.00001)
    
#     for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(l2.get_weights())

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(features_train, labels_train, epochs = 5, batch_size=batch_size,
             validation_data = (features_valid, labels_valid))
    return model


In [26]:
# models = ""
models = [fit_model() for i in range(6)]


Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 23s - loss: 0.1431 - acc: 0.9459 - val_loss: 0.0658 - val_acc: 0.9752
Epoch 2/5
22500/22500 [==============================] - 23s - loss: 0.0313 - acc: 0.9886 - val_loss: 0.0660 - val_acc: 0.9740
Epoch 3/5
22500/22500 [==============================] - 23s - loss: 0.0148 - acc: 0.9950 - val_loss: 0.0718 - val_acc: 0.9728
Epoch 4/5
22500/22500 [==============================] - 23s - loss: 0.0071 - acc: 0.9984 - val_loss: 0.0763 - val_acc: 0.9732
Epoch 5/5
22500/22500 [==============================] - 23s - loss: 0.0059 - acc: 0.9984 - val_loss: 0.0685 - val_acc: 0.9740
Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 23s - loss: 0.1436 - acc: 0.9451 - val_loss: 0.0770 - val_acc: 0.9684
Epoch 2/5
22500/22500 [==============================] - 23s - loss: 0.0330 - acc: 0.9872 - val_loss: 0.0730 - val_acc: 0.9724
Epoch 3/5
225

In [27]:
evals = np.array([m.evaluate(features_valid, labels_valid) for m in models])

2432/2500 [============================>.] - ETA: 0s

In [28]:
print(evals.mean(axis=0))

[ 0.0736  0.9748]


In [29]:
all_preds = np.stack([m.predict(features_test) for m in models])

In [30]:
print(all_preds.shape)

(6, 12500, 2)


In [31]:
avg_preds = all_preds.mean(axis=0)
# avg_preds = all_preds[0]

In [32]:
isdog = avg_preds[:,1]
print(isdog[:5])

[  9.9839e-01   2.5321e-07   9.9969e-01   9.9999e-01   2.2903e-10]


In [33]:
margin = 0.04
isdog = isdog.clip(min=margin, max=1-margin)
print(isdog[:5])

[ 0.96  0.04  0.96  0.96  0.04]


In [34]:
filenames = batches_test.filenames
print(filenames[:5])


['unknown/2212.jpg', 'unknown/11615.jpg', 'unknown/9910.jpg', 'unknown/6117.jpg', 'unknown/541.jpg']


In [35]:
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
print(ids)

[ 2212 11615  9910 ...,     6  7220  4813]


In [36]:
subm = np.stack([ids,isdog], axis=1)
subm[:5]


array([[  2.2120e+03,   9.6000e-01],
       [  1.1615e+04,   4.0000e-02],
       [  9.9100e+03,   9.6000e-01],
       [  6.1170e+03,   9.6000e-01],
       [  5.4100e+02,   4.0000e-02]])

In [37]:
submission_file_name = 'submission1.csv'
np.savetxt(model_path + submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')